In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [8]:
data1 = [4, 9, 87, 
         80 , 2, 18, 
         35, 30, 35, 
         25, 40, 35]

prod  = ['MILK', 'BUTTER', 'CHEESE_1', 'CHEESE_2']
material  = ['FAT', 'DRY', 'WATER']

compos = dict(zip(it.product(prod, material),   # changed to %
                 [d/100 for d in data1]))
deman = dict(zip(prod, [d/1000 for d in [4820, 320, 210, 70]]))
price = dict(zip(prod, [d/1000 for d in [297, 720, 1050, 815]]))
elast = dict(zip(prod+['CHEESE_1CHEESE_2', 'CHEESE_2CHEESE_1'], 
                 [0.4, 2.7, 1.1, 0.4, 0.1, 0.4]))
prev_con = sum(price[i]*deman[i] for i in prod)

In [9]:
model = gp.Model('Agricultural Pricing')

# add vars
x = model.addVars(prod,
                  name='deman')
p = model.addVars(prod,
                  name='price')
model.update()

# objective function
model.setObjective((gp.quicksum(x[i]*p[i] for i in prod)),
                    GRB.MAXIMIZE)

# add constraints

    # products quantity limitation
model.addConstr((gp.quicksum(compos[i,'FAT']*x[i] for i in prod)
                  <= 600),
               name='FAT_limit')
model.addConstr((gp.quicksum(compos[i,'DRY']*x[i] for i in prod)
                  <= 750),
               name='DRY_limit')

    # keep prices index smaller than last year
model.addConstr((gp.quicksum(deman[i]*p[i] for i in prod)
                 <= prev_con),
               name='price_index')

    # elasticity relations
for i in ['MILK', 'BUTTER']:
    model.addConstr(((x[i]-deman[i])/deman[i] 
                     == -elast[i]*(p[i]-price[i])/price[i]),
                    name='elast_'+i)
    
for i,j in [('CHEESE_1', 'CHEESE_2'), ('CHEESE_2', 'CHEESE_1')]:
    model.addConstr(((x[i]-deman[i])/deman[i] 
                     == -elast[i]*(p[i]-price[i])/price[i]
                        + elast[i+j]*(p[j]-price[j])/price[j]),
                    name='elast_'+i)    
    
model.update()
model.write('Agricultural Pricing.lp')
model.params.nonConvex = 2
model.optimize()

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 8 columns and 22 nonzeros
Model fingerprint: 0x3cc101e8
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 13 columns, 36 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 2.791495e+00, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [16]:
report = pd.DataFrame([], 
                      columns=['Demand (tons)', 'Price (per ton)'], 
                      index=prod)
for i in prod:
    report['Demand (tons)'][i] = x[i].x*1e6
    report['Price (per ton)'][i] = p[i].x*1000

print('Revenue of ${:.0f} millions'.format(model.ObjVal*1000))
report

Revenue of $2067 millions


,Demand (tons),Price (per ton)
MILK,4659699.860066,321.693538
BUTTER,677312.417363,422.239652
CHEESE_1,266099.015815,831.912667
CHEESE_2,53357.934253,1130.125675
